In [ ]:
!pip install -q transformers torch scikit-learn


In [ ]:
import pandas as pd
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:

# Dataset B — labeled
train_df = pd.read_csv("/content/Arabic.csv")
train_df = train_df.dropna(subset=["tweet", "label"])

# Dataset A — unlabeled
real_df = pd.read_csv("/content/merged_twitterdata.csv")
real_df = real_df.dropna(subset=["text"])


## Requirement Analysis & Data Organization




In [ ]:
label_map = {
    "not": 0,
    "offensive": 1
}

train_df["label"] = train_df["label"].astype(str).str.strip().str.lower()
train_df["label_id"] = train_df["label"].map(label_map)

print(train_df[["label", "label_id"]].head())
print("NaN in label_id:", train_df["label_id"].isna().sum())
print("Label counts:\n", train_df["label"].value_counts())


# Train–Validation–Test Split to Reduce Overfitting


In [ ]:
from sklearn.model_selection import train_test_split

X = train_df["tweet"].values
y = train_df["label_id"].values

# Train (70%) + Temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

# Validation (15%) + Test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42,
    stratify=y_temp
)

print("Train size:", len(X_train))
print("Val size:", len(X_val))
print("Test size:", len(X_test))

print("\nClass distribution:")
import numpy as np
print("Train:", np.bincount(y_train))
print("Val:", np.bincount(y_val))
print("Test:", np.bincount(y_test))


# Setup LLM Environment and Load Pre-trained Model



In [ ]:
!pip install -q transformers torch scikit-learn


In [ ]:
import warnings
warnings.filterwarnings("ignore")

print("Environment ready: Transformers, Torch, and Scikit-learn")


**Reasoning**:
Now that the necessary libraries are installed, the next step is to import `AutoTokenizer` and `AutoModelForSequenceClassification` from the `transformers` library and then load a pre-trained Arabic transformer model and its tokenizer, as per the subtask instructions.



# Load Pre-trained Arabic BERT Model and Tokenizer


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Choose a pre-trained Arabic BERT model
model_name = "aubmindlab/bert-base-arabertv2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Tokenizer for {model_name} loaded successfully.")

# Load model for binary sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)
print(f"Model for {model_name} loaded successfully.")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model moved to device: {device}")


# Handle Class Imbalance (Train Only)


In [ ]:
import torch
from torch.utils.data import WeightedRandomSampler
import numpy as np

# Compute sample weights for the training set only
class_counts = np.bincount(y_train)              # e.g., [not_count, offensive_count]
class_weights = 1.0 / class_counts               # higher weight for minority class
sample_weights = class_weights[y_train]          # weight per training sample

train_sampler = WeightedRandomSampler(
    weights=torch.tensor(sample_weights, dtype=torch.double),
    num_samples=len(sample_weights),
    replacement=True
)

print("Train class counts:", class_counts)
print("Train class weights:", class_weights)
print("Balanced sampler ready ")


# Tokenize Data and Build Datasets


In [ ]:
from torch.utils.data import TensorDataset

max_length = 128

# Tokenize Train
train_encodings = tokenizer(
    list(X_train),
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Tokenize Validation
val_encodings = tokenizer(
    list(X_val),
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Tokenize Test
test_encodings = tokenizer(
    list(X_test),
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# Convert labels to tensors
y_train_t = torch.tensor(y_train, dtype=torch.long)
y_val_t   = torch.tensor(y_val,   dtype=torch.long)
y_test_t  = torch.tensor(y_test,  dtype=torch.long)

# Build datasets
train_dataset = TensorDataset(
    train_encodings["input_ids"],
    train_encodings["attention_mask"],
    y_train_t
)

val_dataset = TensorDataset(
    val_encodings["input_ids"],
    val_encodings["attention_mask"],
    y_val_t
)

test_dataset = TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
    y_test_t
)

print("Datasets ready:",
      len(train_dataset),
      len(val_dataset),
      len(test_dataset))


# Create DataLoaders


In [ ]:
from torch.utils.data import DataLoader

batch_size = 16

# Train loader with balanced sampler
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    sampler=train_sampler
)

# Validation loader
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False
)

# Test loader
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

print("DataLoaders ready:",
      len(train_loader),
      len(val_loader),
      len(test_loader))


# Training Loop with Validation


In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 3

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # ===== Training =====
    model.train()
    train_loss = 0.0

    for batch in tqdm(train_loader, desc="Training"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Average training loss: {avg_train_loss:.4f}")

    # ===== Validation =====
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            preds = torch.argmax(logits, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total

    print(f"Validation loss: {avg_val_loss:.4f}")
    print(f"Validation accuracy: {val_accuracy:.4f}")


# Final Evaluation on Test Set


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Accuracy
test_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Test Accuracy: {test_accuracy:.4f}")

# Detailed metrics
print("\nClassification Report:")
print(classification_report(
    all_labels,
    all_preds,
    target_names=["not", "offensive"]
))

print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


# Optional: Inference on Unlabeled Data (real_df)


## Tokenize Unlabeled Data


In [ ]:
import torch

max_length = 128  # keep consistent with training

real_texts = real_df["text"].astype(str).tolist()

real_encodings = tokenizer(
    real_texts,
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

print("Real data tokenized ")
print("input_ids:", real_encodings["input_ids"].shape)
print("attention_mask:", real_encodings["attention_mask"].shape)


## Predict Labels for real_df


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

model.eval()

real_dataset = TensorDataset(
    real_encodings["input_ids"],
    real_encodings["attention_mask"]
)

real_loader = DataLoader(real_dataset, batch_size=32, shuffle=False)

all_real_preds = []

with torch.no_grad():
    for batch in real_loader:
        input_ids, attention_mask = [b.to(device) for b in batch]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_real_preds.extend(preds.cpu().numpy())

# Map numeric predictions back to label names
id2label = {0: "not", 1: "offensive"}
real_df["pred_id"] = all_real_preds
real_df["pred_label"] = real_df["pred_id"].map(id2label)

print("Predictions added ")
print(real_df[["text", "pred_label"]].head())
print("\nPred label counts:")
print(real_df["pred_label"].value_counts())


# Human-Labeled Dataset (Manual Annotation)

## Load and Inspect Human-Labeled Twitter Data

In [ ]:
human_df = pd.read_csv(
    "/content/merged_twitterdata with human classification.csv",
    sep=";",
    encoding="utf-8-sig",
    engine="python"
)

print(human_df.columns)
print(human_df.shape)
human_df.head()


## Inspect Human Label Distribution

In [ ]:
human_df["classification"] = human_df["classification"].astype(str).str.strip().str.lower()
print(human_df["classification"].value_counts())


# Model vs Human Comparison

## Generate Model Predictions for Human-Labeled Tweets

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

max_length = 128

texts = human_df["text"].astype(str).tolist()

enc = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

dataset = TensorDataset(enc["input_ids"], enc["attention_mask"])
loader = DataLoader(dataset, batch_size=32, shuffle=False)

model.eval()
preds = []

with torch.no_grad():
    for batch in loader:
        input_ids, attention_mask = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        batch_preds = torch.argmax(outputs.logits, dim=1)
        preds.extend(batch_preds.cpu().numpy())

id2label = {0: "not", 1: "offensive"}
human_df["model_pred"] = [id2label[p] for p in preds]

print(human_df[["text", "classification", "model_pred"]].head(10))
print("\nModel prediction counts:")
print(human_df["model_pred"].value_counts())


# Model Evaluation on Full Human-Labeled Dataset (Unbalanced Test)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import torch

# Encode human-labeled texts (unbalanced)
human_enc = tokenizer(
    human_df["text"].astype(str).tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Convert labels
label_map = {"not": 0, "offensive": 1}
y_human = torch.tensor(
    human_df["classification"].map(label_map).values
)

# Create dataset and loader
human_dataset = TensorDataset(
    human_enc["input_ids"],
    human_enc["attention_mask"],
    y_human
)

human_loader = DataLoader(
    human_dataset,
    batch_size=16,
    shuffle=False
)

# Evaluation
model.eval()
all_preds = []
all_true = []

with torch.no_grad():
    for batch in human_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_true.extend(labels.cpu().numpy())

print("Unbalanced Human Test Accuracy:",
      accuracy_score(all_true, all_preds))

print("\nClassification Report (Human Unbalanced):")
print(classification_report(all_true, all_preds,
                            target_names=["not", "offensive"]))

print("\nConfusion Matrix:")
print(confusion_matrix(all_true, all_preds))


# Model vs Human Annotation Evaluation

## Confusion Matrix and Classification Report (Human vs Model)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = human_df["classification"]
y_pred = human_df["model_pred"]

print("Classification Report (Human vs Model):")
print(classification_report(
    y_true,
    y_pred,
    target_names=["not", "offensive"]
))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


In [ ]:
save_dir = "/content/arabert_abuse_model"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved to:", save_dir)


In [ ]:
!zip -r arabert_abuse_model.zip /content/arabert_abuse_model


# Human-Labeled Dataset as Test Set Only

### Create a balanced TEST set from human_df

In [ ]:
import pandas as pd

# Work on a clean copy
test_df = human_df.copy()

# Basic cleaning (ensure correct labels)
test_df["classification"] = test_df["classification"].astype(str).str.strip().str.lower()

# Balance the test set: take the same number from each class
min_n = test_df["classification"].value_counts().min()

balanced_test_df = (
    test_df.groupby("classification", group_keys=False)
           .apply(lambda x: x.sample(n=min_n, random_state=42))
           .sample(frac=1, random_state=42)   # shuffle
           .reset_index(drop=True)
)

print("Original test distribution:")
print(test_df["classification"].value_counts())

print("\nBalanced test distribution:")
print(balanced_test_df["classification"].value_counts())

print("\nBalanced test size:", balanced_test_df.shape)
balanced_test_df.head()


# Model Evaluation on Balanced Human-Labeled Test Set

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Encode balanced test texts
test_enc = tokenizer(
    balanced_test_df["text"].astype(str).tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Convert labels to numeric
label_map = {"not": 0, "offensive": 1}
y_test = torch.tensor(
    balanced_test_df["classification"].map(label_map).values
)

# Create test dataset & loader
test_dataset = TensorDataset(
    test_enc["input_ids"],
    test_enc["attention_mask"],
    y_test
)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Evaluation
model.eval()
all_preds = []
all_true = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_true.extend(labels.cpu().numpy())

# Metrics
print("Balanced Test Accuracy:", accuracy_score(all_true, all_preds))
print("\nClassification Report:")
print(classification_report(all_true, all_preds, target_names=["not", "offensive"]))

print("\nConfusion Matrix:")
print(confusion_matrix(all_true, all_preds))


In [ ]:
# ===============================
# Save evaluation results (TXT + CSV)
# ===============================

import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Convert lists to arrays (assumes all_true & all_preds already exist)
y_true = all_true
y_pred = all_preds

# -------- TXT report --------
txt_path = "balanced_test_results.txt"

with open(txt_path, "w", encoding="utf-8") as f:
    f.write("Balanced Human-Labeled Test Results\n")
    f.write("=" * 40 + "\n\n")

    f.write(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}\n\n")

    f.write("Classification Report:\n")
    f.write(
        classification_report(
            y_true,
            y_pred,
            target_names=["not", "offensive"]
        )
    )
    f.write("\n\nConfusion Matrix:\n")
    f.write(str(confusion_matrix(y_true, y_pred)))

print(f"TXT results saved to: {txt_path}")

# -------- CSV predictions --------
csv_path = "balanced_test_predictions.csv"

results_df = balanced_test_df.copy()
results_df["true_label"] = results_df["classification"]
results_df["pred_label"] = ["not" if p == 0 else "offensive" for p in y_pred]

results_df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"CSV predictions saved to: {csv_path}")

results_df.head()
